In [1]:
#Импорт библиотек
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup # Превращалка html в текст.
import pymorphy2 # Морфологический анализатор.
from collections import Counter # Не считать же частоты самим.
import re # Регулярные выражения.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import roc_auc_score, roc_curve #импорт roc и auc roc
from sklearn.metrics.pairwise import cosine_similarity
import math
from timeit import default_timer as timer
import pymorphy2 # Морфологический анализатор.
morph=pymorphy2.MorphAnalyzer()

# Создание функций для решения задачи

In [4]:
def GetClass(name, descr):
    cosname = dict.fromkeys(name.split(), 1)
    cosdescr = dict.fromkeys(descr.split(), 1)

    maxsim1 = 0
    maxsim2 = 0
    maxsim3 = 0
    maxsim4 = 0

    
    for i in vt1d:
    
        cursim = cosineSimilarity(cosname, i)
    
        if maxsim1 < cursim:
            maxsim1 = cursim
            
        if maxsim1==1:
            return 1
        
    if maxsim1==1:
        return 1

    
    for i in vt2d:
    
        cursim = cosineSimilarity(cosname, i)
    
        if maxsim2 < cursim:
            maxsim2 = cursim
        
        if maxsim2 >= maxsim1:
            break
            
    if maxsim1 > maxsim2 or maxsim1==1:
        return 1
    elif maxsim1 < maxsim2 or maxsim1==0:
        return 0
    else:
        
        for i in vt3d:
    
            cursim = cosineSimilarity(cosdescr, i)
    
            if maxsim3 < cursim:
                maxsim3 = cursim

        for i in vt4d:
    
            cursim = cosineSimilarity(cosdescr, i)
    
            if maxsim4 < cursim:
                maxsim4 = cursim
                
            if maxsim4 >= maxsim3:
                break
        
        if maxsim3 > maxsim4:
            return 1
        else: 
            return 0


In [5]:
#функция для выделения описания должности из html-текста
def DescHTMLtoTEXT(htmltext):
    bs=BeautifulSoup(htmltext, "html5lib").body

    text = BeautifulSoup(htmltext, 'lxml').get_text()
    
    return text
    

In [6]:
def cosineSimilarity(a, b):
    if len(a.keys())==0 or len(b.keys())==0:
        return 0
    sumab=sum([a[na]*b[na] for na in a.keys() if na in b.keys()])
    suma2=sum([a[na]*a[na] for na in a.keys()])
    sumb2=sum([b[nb]*b[nb] for nb in b.keys()])
    return sumab/math.sqrt(suma2*sumb2)

In [7]:
def GetMeaningWords(string):
    stst = ''
    
    tokensrus = re.findall('[А-Яа-яЁё]+', string)
    tokenslatin = re.findall('[A-Za-z]+', string)
    
    for t in tokensrus:
        if len(t) > 2:
        
            pv = morph.parse(t)
            for p in pv:
                if p.tag.POS in ['ADJF', 'NOUN', 'VERB', 'PRTF']:
                    stst = stst + p.normal_form + ' '
                    break  
    for t in tokenslatin:
        stst = stst + t + ' '
    return stst

In [8]:
def GetMeanDictionaries(input_dataframe):
    t = timer()
    #Разделение датафрейма на вакансии интересующие и неинтересующие
    target1 = input_dataframe[input_dataframe.target==1]
    target0 = input_dataframe[input_dataframe.target==0]
    
    #elapsed = timer() - t
    #print('Target: ', elapsed)
    #t = timer()
        
    #Получение всех слов в наименованиях и описаниях вакансий
    Names_target1 = ' '.join(target1.loc[:,'name'].apply(GetMeaningWords))
    Names_target0 = ' '.join(target0.loc[:,'name'].apply(GetMeaningWords))
    
    #elapsed = timer() - t
    #print('Words: names: ', elapsed)
    #t = timer()
       
    Desc_target1 = ' '.join(target1.loc[:,'description'].apply(GetMeaningWords))
    Desc_target0 = ' '.join(target0.loc[:,'description'].apply(GetMeaningWords))
    
    #elapsed = timer() - t
    #print('Words: descriptions:', elapsed)
    #t = timer()
    
    #CountVectorizer для наименования вакансий
    counter_vec    = CountVectorizer(ngram_range=(2,3))
    analyzer       = counter_vec.build_analyzer()
    
    #наименования вакансий интересующих
    if len(Names_target1) > 0:
        result         = counter_vec.fit_transform([Names_target1])
        words_target1  = [x[0] for x in sorted(counter_vec.vocabulary_.items(), key=lambda x: x[1])]
        vocabulary_target1 = pd.DataFrame({'words': words_target1, 'freq': result.toarray()[0]})
    else:
        vocabulary_target1 = pd.DataFrame(columns=['words','freq'])
    #vocabulary_target1 = vocabulary_target1[vocabulary_target1.freq > 3].sort_values('freq',ascending=False)
    
    #elapsed = timer() - t
    #print('CountVectorizer names target=1: ', elapsed)
    #t = timer()
    
    #наименования вакансий неинтересующих
    if len(Names_target0) > 0:
        #print(len(Names_target0), Names_target0)
        try:
            result         = counter_vec.fit_transform([Names_target0])
            words_target0  = [x[0] for x in sorted(counter_vec.vocabulary_.items(), key=lambda x: x[1])]
            vocabulary_target0 = pd.DataFrame({'words': words_target0, 'freq': result.toarray()[0]})
        except:
            vocabulary_target0 = pd.DataFrame(columns=['words','freq'])
    else:
        vocabulary_target0 = pd.DataFrame(columns=['words','freq'])
    #vocabulary_target0 = vocabulary_target0[vocabulary_target0.freq > 3].sort_values('freq',ascending=False)
     
    #elapsed = timer() - t
    #print('CountVectorizer names target=0: ', elapsed)
    #t = timer()

    #описания вакансий интересующих 
    if len(Desc_target1) > 0:
        result  = counter_vec.fit_transform([Desc_target1])
        words_target1_desc = [x[0] for x in sorted(counter_vec.vocabulary_.items(), key=lambda x: x[1])]
        vocabulary_target1_desc = pd.DataFrame({'words': words_target1_desc, 'freq': result.toarray()[0]})
    else:
        vocabulary_target1_desc = pd.DataFrame(columns=['words','freq'])
    #vocabulary_target1_desc = vocabulary_target1_desc[vocabulary_target1_desc.freq > 3].sort_values('freq',ascending=False)
    
    #elapsed = timer() - t
    #print('CountVectorizer descriptions target=1: ', elapsed)
    #t = timer()
    if len(Desc_target0) > 0:
        result  = counter_vec.fit_transform([Desc_target0])
        words_target0_desc = [x[0] for x in sorted(counter_vec.vocabulary_.items(), key=lambda x: x[1])]
        vocabulary_target0_desc = pd.DataFrame({'words': words_target0_desc, 'freq': result.toarray()[0]})
    else:
        vocabulary_target0_desc = pd.DataFrame(columns=['words','freq'])
    #vocabulary_target0_desc = vocabulary_target0_desc[vocabulary_target0_desc.freq > 3].sort_values('freq',ascending=False)

    #elapsed = timer() - t
    #print('CountVectorizer descriptions target=0: ', elapsed)
    #t = timer()

    #Возвращаю датафреймы с колонками "words" и "freq"
    return vocabulary_target1, vocabulary_target0, vocabulary_target1_desc, vocabulary_target0_desc

In [9]:
def DeleteSimilarWords(frame_target1, frame_target0):
    #Удаление повторяющихся слов в словарях
    words_vt1 = frame_target1['words']
    words_vt2 = frame_target0['words']
    words2list = words_vt2.tolist()

    for index1, word in enumerate(words_vt1):
        if word in words2list:
            frame_target1 = frame_target1.drop([words_vt1.index[index1]])
            for index2, word2 in enumerate(words_vt2):
                if word == word2:
                    frame_target0 = frame_target0.drop(words_vt2.index[index2])
                    break

In [10]:
def toDict(v1,v2,v3,v4):
    #не списки, а словари, для измерения косинусовых расстояний
    vt1d = []
    for i in v1:
        ii = i.split()
        vt1d.append(dict.fromkeys(ii,1))
    
    vt2d = []
    for i in v2:
        ii = i.split()
        vt2d.append(dict.fromkeys(ii,1))
    
    vt3d = []
    for i in v3:
        ii = i.split()
        vt3d.append(dict.fromkeys(ii,1))
    
    vt4d = []
    for i in v4:
        ii = i.split()
        vt4d.append(dict.fromkeys(ii,1))
                    
    return vt1d, vt2d, vt3d, vt4d

# Здесь начинается решение задачи

In [104]:
#Чтение файла
dataframe = pd.read_csv('train.csv', sep = '\t')

In [66]:
#порционное получение словарей
Names_target1_vocabulary = pd.DataFrame(columns=['words','freq'])
Names_target0_vocabulary = pd.DataFrame(columns=['words','freq'])
Descr_target1_vocabulary = pd.DataFrame(columns=['words','freq'])
Descr_target0_vocabulary = pd.DataFrame(columns=['words','freq'])

try:
    protokol_file = open('protokol.txt', 'r')
    asd = protokol_file.read()
    listsrt = asd.split('\n')

    start = int(listsrt[len(listsrt)-2])
    protokol_file.close()
except:
    start = 0

for i in range(start, len(dataframe), 1000):
    portion = dataframe.loc[i:i+999,:].copy()
    portion['description'] = portion['description'].map(DescHTMLtoTEXT) #по части
    
    n1,n0,d1,d0 = GetMeanDictionaries(portion)
    
    Names_target1_vocabulary = pd.concat([Names_target1_vocabulary, n1])
    Names_target0_vocabulary = pd.concat([Names_target0_vocabulary, n0])
    Descr_target1_vocabulary = pd.concat([Descr_target1_vocabulary, d1])
    Descr_target0_vocabulary = pd.concat([Descr_target0_vocabulary, d0])

    Names_target1_vocabulary.to_csv('Names_target1.csv', index = False)
    Names_target0_vocabulary.to_csv('Names_target0.csv', index = False)
    Descr_target1_vocabulary.to_csv('Descr_target1.csv', index = False)
    Descr_target0_vocabulary.to_csv('Descr_target0.csv', index = False)
    
    protokol_file = open('protokol.txt', 'w')
    protokol_file.write(str(i+99))
    protokol_file.write('\n')   
    protokol_file.close()    

In [11]:
#Чтение из собранного ранее словаря
vt1 = pd.read_csv('Names_target1.csv')
vt1 = vt1.groupby('words').sum()
vt1 = vt1[vt1.freq > 20]
vt1.reset_index(inplace=True)

vt2 = pd.read_csv('Names_target0.csv')
vt2 = vt2.groupby('words').sum()
vt2 = vt2[vt2.freq > 20]
vt2.reset_index(inplace=True)

vt3 = pd.read_csv('Descr_target1.csv')
vt3 = vt3.groupby('words').sum()
vt3 = vt3[vt3.freq > 20]
vt3.reset_index(inplace=True)

vt4 = pd.read_csv('Descr_target0.csv')
vt4 = vt4.groupby('words').sum()
vt4 = vt4[vt4.freq > 20]
vt4.reset_index(inplace=True)


In [18]:
vt1d, vt2d, vt3d, vt4d = toDict(vt1['words'].tolist(),vt2['words'].tolist(),vt3['words'].tolist(),vt4['words'].tolist())

In [153]:
#Подготовка тестовой выборки
testdf =  dataframe.loc[0:100,:].copy() #Для отладки, чтобы не ждать по 15 минут
#testdf = dataframe_less.copy()
testdf['description'] = testdf['description'].map(DescHTMLtoTEXT) #по части

testdf['name1'] = testdf['name'].apply(lambda x: ' '.join(a[0] for a in re.findall("([А-ЯЁа-яёA-Za-z]+(-[А-ЯЁа-яёA-Za-z]+)*)", x)))
testdf['name1'] = testdf['name'].map(GetMeaningWords)
testdf['description1'] = testdf['description'].map(GetMeaningWords)

In [244]:
#Предсказание 
testdf['newtarget'] = testdf.apply(lambda x: GetClassNew(x['name1'], x['description1']), axis = 1)

In [245]:
#подсчет roc_auc_score
print('roc_auc модели ', roc_auc_score(testdf['target'], testdf['newtarget']))
print (testdf.shape)

roc_auc модели DecisionTreeClassifier 0.6436758893280632
(101, 7)


In [246]:
#для анализа ошибок модели
print('False negative:', testdf[(testdf.target!=testdf.newtarget) & (testdf.newtarget==0)].shape)
print('False positive:', testdf[(testdf.target!=testdf.newtarget) & (testdf.newtarget==1)].shape)

False negative: (1, 7)
False positive: (38, 7)


In [232]:
errors = testdf[(testdf.target!=testdf.newtarget) & (testdf.newtarget==0)]
#GetClass(errors['name1'][8], errors['description1'][8])
errors

,id,name,description,target,name1,description1,newtarget
64,64,Бариста,"Мы предлагаем: Место работы: уютное кафе м. ""...",1,барист,предлагать место работа уютный кафе метр курск...,0


# Тестовые данные

In [49]:
#Чтение файла
dataframe_test = pd.read_csv('test.csv', sep = '\t')

In [51]:
dataframe_test['description'] = dataframe_test['description'].map(DescHTMLtoTEXT) #по части

dataframe_test['name1'] = dataframe_test['name'].apply(lambda x: ' '.join(a[0] for a in re.findall("([А-ЯЁа-яёA-Za-z]+(-[А-ЯЁа-яёA-Za-z]+)*)", x)))
dataframe_test['name1'] = dataframe_test['name'].map(GetMeaningWords)
dataframe_test['description1'] = dataframe_test['description'].map(GetMeaningWords)

In [15]:
dataframe_test.to_csv('norm.csv', index=None)

In [ ]:
dataframe_test = pd.read_csv('norm.csv')

In [16]:
dataframe_test = dataframe_test.fillna(' ')


In [21]:
resultDF = pd.DataFrame(columns = ['id', 'target'])

try:
    protokol_file = open('protokol.txt', 'r')
    asd = protokol_file.read()
    listsrt = asd.split('\n')

    start = int(listsrt[len(listsrt)-2]) +1
    protokol_file.close()
except:
    start = 0

for i in range(start, 130000, 500):
    dataframe_test_portion = dataframe_test.loc[i:i+499,:]
    dataframe_test_portion['target'] = dataframe_test_portion.apply(lambda x: GetClass(x['name1'], x['description1']), axis = 1)
    
    resultDF = pd.concat([resultDF, dataframe_test_portion])
    resultDF.to_csv('predict.csv', index=None,columns=['id','name','target'])
    protokol_file = open('protokol.txt', 'w')
    protokol_file.write(str(i+499))
    protokol_file.write('\n')   
    protokol_file.close() 

C:\Users\gavrilov_ej\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\gavrilov_ej\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

